In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#import datetime

import talib as ta

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
#from mlxtend.classifier import StackingCVClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [3]:
DfMain = pd.read_csv("C:/Users/marli/Downloads/FinalStocksDFHighResolutionMin.csv")

In [4]:
# Importing libraries
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
#from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
import talib as ta
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit
from  sklearn.ensemble import GradientBoostingClassifier
import multiprocessing
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score,precision_score,recall_score

In [ ]:
def DataPrep(DF,Stock):
    Df = DF[DF["Symbol"]==Stock]
    Df = Df.rename(columns={'Datetime': 'Time'}) 
    Df = Df.drop(Df[Df['Volume'] == 0].index)
    Df['Time'] = pd.to_datetime(Df['Time'])
    Df = Df.drop(["IPO Year"], axis=1)
    Df.set_index('Time', inplace=True)
    n = 10
    Df['RSI'] = ta.RSI(np.array(Df['Close'].shift(1)), timeperiod=n)
    Df['SMA'] = Df['Close'].shift(1).rolling(window=n).mean()
    Df['Corr'] = Df['Close'].shift(1).rolling(window=n).corr(Df['SMA'].shift(1))
    Df['SAR'] = ta.SAR(np.array(Df['High'].shift(1)),
                   np.array(Df['Low'].shift(1)), 0.2, 0.2)
    Df['ADX'] = ta.ADX(np.array(Df['High'].shift(1)), np.array(
    Df['Low'].shift(1)), np.array(Df['Open']), timeperiod=n)
    Df['OpenShifted'] = Df['Open'].shift(1)
    Df['CloseShifted'] = Df['Close'].shift(1)
    Df['HighShifted'] = Df['High'].shift(1)
    Df['LowShifted'] = Df['Low'].shift(1)
    Df['OO'] = Df['Open']-Df['OpenShifted']
    Df['OC'] = Df['Open']-Df['CloseShifted']
    Df['Ret'] = (Df['Open'].shift(-1)-Df['Open'])/Df['Open']
    for i in range(1, n):
        Df['return%i' % i] = Df['Ret'].shift(i)
    #Trimming the dataDF
    Df.loc[Df['Corr'] < -1, 'Corr'] = -1
    Df.loc[Df['Corr'] > 1, 'Corr'] = 1
    Df = Df.dropna()
    return(Df)

In [ ]:
def XYPrep(DF,t=.8):
    split = int(t*len(DF))
    Df = DF
    #Creating output signal
    Df['Signal'] = 0#
    Df.loc[Df['Ret'] > Df['Ret'][:split].quantile(q=0.66), 'Signal'] = -1
    Df.loc[Df['Ret'] < Df['Ret'][:split].quantile(q=0.33), 'Signal'] = 1
    #Df.loc[Df['Ret'] > Df['Ret'][:split].quantile(q=0.8), 'Signal'] = 2
    #Df.loc[Df['Ret'] < Df['Ret'][:split].quantile(q=0.2), 'Signal'] = -2
    
    # Assign a value of 0 to 'Signal' column at 1529 time
    Df.loc[(Df.index.hour == 15) & (Df.index.minute == 29), 'Signal'] = 0
    # Assign a value of 0 to 'Fut_Ret' column at 1529 time
    #Df.loc[(Df.index.hour == 15) & (Df.index.minute == 29), 'Fut_Ret'] = 0
    split = int(t*len(Df))
    TestDf = Df.iloc[split:]
    TrainDf = Df.iloc[:split]
    return(TrainDf,TestDf)

def DataSplit(DF):
    # Creating the features and values
    X = DF.drop(['Close', 'Signal', 'High','Open',
            'Low', 'Ret',"Name","Dividends", "Country",'Market Cap','Volume', "Symbol",
             "Sector","Industry","Stock Splits"], axis=1)
    Y = DF['Signal']
    Ret = DF['Ret']
    return(X,Y,Ret)

In [ ]:
def TrainFit(X_Train,Y_Train,Model,parameters):

    steps = [('scaler', StandardScaler()), ('Model', Model)]
    pipeline = Pipeline(steps)

    clf = GridSearchCV(pipeline,parameters, cv=5 , verbose=0,n_jobs=1,scoring = "f1_macro")#Not sure if this is best metric
    clf.fit(X_Train, Y_Train)
    #print(clf.best_score_)
    #print(clf.best_params_)

    pipeline.set_params(**clf.best_params_)
    pipeline.fit(X_Train, Y_Train)
    return(pipeline)

def TestModel(Mod,X_Test,Y_Test,Ret_Test,Name = "Model"):
    predictions = Mod.predict(X_Test)
    #cm = confusion_matrix(Y_Test, predictions, labels=Mod.classes_)

    #disp = ConfusionMatrixDisplay(confusion_matrix=cm,
    #                           display_labels=Mod.classes_)
    #disp.plot()
    StragyReturn = Ret_Test*predictions
    accuracy = accuracy_score(Y_Test, predictions)          
    recall = recall_score(Y_Test,predictions,average=None)
    precision = precision_score(Y_Test,predictions,average=None)
    DF = pd.DataFrame(np.array([[StragyReturn.mean(), recall[0], recall[1],recall[1]
                                 , precision[0], precision[1],precision[1],accuracy]]),
                   columns=[f"{Name} StragyReturn", f"{Name} recall -1", f"{Name} recall 0",f"{Name} recall 1",
                            f"{Name} precision -1 ",f"{Name} precision 0",f"{Name} precision 1",f"{Name} accuracy"])
    return(DF)

In [5]:
def Driver(DfMain,StockName):
    Df = DataPrep(DfMain, StockName)
    TrainDf,TestDf = XYPrep(Df)
    X_Test,Y_Test,Ret_Test = DataSplit(TestDf)
    X_Train,Y_Train,Ret_Train = DataSplit(TrainDf)
    TrainedModel = TrainFit(X_Train,Y_Train,Model = GradientBoostingClassifier(), parameters = {'Model__max_depth': [1,2,4], 
              'Model__n_estimators': [50, 100],
               'Model__min_weight_fraction_leaf': [.1],
              'Model__learning_rate': [.9]})
    Results1=TestModel(TrainedModel,X_Test,Y_Test,Ret_Test,Name = "XGB")
    TrainedModel2 = TrainFit(X_Train,Y_Train,Model = RandomForestClassifier(), 
                             parameters = {'Model__n_estimators': [100, 200,400]})
    Results2=TestModel(TrainedModel2,X_Test,Y_Test,Ret_Test,Name = "RandomForest")
    
    TrainedModel3 = TrainFit(X_Train,Y_Train,Model = KNeighborsClassifier(), 
                             parameters = {'Model__n_neighbors': [1, 5,20]})
    
    Results3 = TestModel(TrainedModel3,X_Test,Y_Test,Ret_Test,Name = "KNN")
    
    TrainedModel4 = TrainFit(X_Train,Y_Train,Model = StackingClassifier(estimators=[('XGB', GradientBoostingClassifier()),
                                                                                    ('KNN', KNeighborsClassifier())],
                                                                        final_estimator=LogisticRegression()), 
                             parameters = {'Model__XGB__max_depth': [1,2,4], 
                            'Model__XGB__n_estimators': [50, 100],
                            'Model__XGB__min_weight_fraction_leaf': [.1],
                            'Model__XGB__learning_rate': [.9],
                            'Model__KNN__n_neighbors': [1, 5,20]})
    
    Results4 = TestModel(TrainedModel4,X_Test,Y_Test,Ret_Test,Name = "Stacking")
    
    
    
    DFMainInfo = Df.head(1)[["Symbol","Name","Market Cap","Country","Sector","Industry"]]
    
    DFMainInfo.reset_index(drop=True, inplace=True)
    ReturnDF = pd.concat([DFMainInfo, Results1, Results2, Results3,Results4], axis=1)
    return(ReturnDF)

In [ ]:
Driver(DfMain, "MSFT")
#Df = DataPrep(DfMain, "MSFT")

In [14]:
#from sklearn.metrics import SCORERS
#sorted(SCORERS.keys())

True

In [ ]:
raise Exception
StockNameDF = pd.read_csv("C:\\Users\\marli\\Downloads\\nasdaq_screener_1637446085891.csv")

StockInfo = StockNameDF[["Symbol","Name", "Market Cap", "Country", "IPO Year","Sector","Industry"]]

StockNames = StockInfo["Symbol"]

NewDF = pd.read_csv("C:\\Users\\marli\\Downloads\\FinalAnalysisMin2.csv")

StockNamesAlreadyDone = list(NewDF["Symbol"])

new_strings = []
for string in StockNames:
    new_string = string.replace("/", "")
    new_strings.append(new_string)
StockNames = new_strings

i=0
try:
    for Name in StockNames:
        if (Name in StockNamesAlreadyDone):
            continue
        #if(i%30!=2):
        #    continue
        print("Started working on {}".format(Name))
        try:
            if(i==0):
                MainDF2 = Driver(DfMain, Name)
                continue
        except Exception as e:
            print(e)
            continue
        i+=1
        print(i)
        try:
            Biz = Driver(DfMain, Name)
        except Exception as e:
            print(e)
            continue
        print(Biz[["Symbol","Stacking accuracy"]])
        MainDF2 = pd.concat([MainDF2, Biz], axis=0)
except Exception as e :
    print(e)
    MainDF2.to_csv("C:\\Users\\marli\\Downloads\\FinalAnalysisMin.csv")
    raise e
MainDF2.to_csv("C:\\Users\\marli\\Downloads\\FinalAnalysisMin.csv")

Started working on AA
Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.
Started working on AAC
Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.
Started working on AACG
Started working on AACI
Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.
Started working on AACIU
inputs are all NaN
Started working on AACIW
Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.
Started working on AADI
Started working on AAIC
Started working on AAIC^B
inputs are all NaN
Started working on AAIC^C
inputs are all NaN
Started working on AAIN
inputs are all NaN
Started working on AAL
Started working on AAMC
Started working on AAME
Started working on AAN
Started working on AAOI
Started working on AAON
Started working on AAP
Started working on AAPL
Started working on AAQC
Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.
Started working on AAT
Started working on AATC
Started working o

In [ ]:
MainDF2.to_csv("C:\\Users\\marli\\Downloads\\FinalAnalysisMin2.csv")

In [ ]:
#split = int(.8*len(Df))
#Df['Ret'].hist(bins=200)
#plt.axvline(x=Df['Ret'][:split].quantile(q=0.66),color='r')
#plt.axvline(x=Df['Ret'][:split].quantile(q=0.33),color='r')
#lt.axvline(x=Df['Ret'][:split].quantile(q=0.8),color='r')
#plt.axvline(x=Df['Ret'][:split].quantile(q=0.2),color='r')
#Y_Test.value_counts()

In [ ]:
#TrainedModel2 = TrainFit(X_Train,Y_Train,Model = SVC(), parameters = {'Model__C': [10, 100, 1000, 10000], 
#                                                'Model__gamma': [1e-2, 1e-1, 1e0], 'Model__kernel': ['rbf']})
#TestModel(TrainedModel2,X_Test,Y_Test,Ret_Test)

In [ ]:
#TrainedModel3 = TrainFit(X_Train,Y_Train,Model = RandomForestClassifier(), parameters = {'Model__n_estimators': [100, 200]})
#TestModel(TrainedModel3,X_Test,Y_Test,Ret_Test)

In [ ]:
#TrainedModel4 = TrainFit(X_Train,Y_Train,Model = XGBClassifier(), parameters = {'Model__random_state': [123], "Model__verbosity":[0], 'Model__use_label_encoder':[False]})
#TestModel(TrainedModel4,X_Test,Y_Test,Ret_Test)

In [ ]:
#TrainedModel5 = TrainFit(X_Train,Y_Train,Model = LogisticRegression(), parameters = {'Model__random_state': [123]})
#TestModel(TrainedModel5,X_Test,Y_Test,Ret_Test)